In [3]:
%pylab inline
import pandas as pd 
import gplearn as gpl    
from gplearn.genetic import SymbolicRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import graphviz

d0 = pd.read_csv("exa1.csv",delimiter=",")
d = np.asarray(d0)
#x3 and x6 are the "right" input variables and should a least be included
X_train = d[:,(3,6,7,9)]              #First subset
#X_train = d[:,(3,5,6,7,8,9)]          #Second subset
y_train = d[:,4]
X_train[:,0] = X_train[:,0]/15000.0  #Scaling this variable to make it on a reasonable scale

from joblib import Parallel, delayed
import multiprocessing
     
inputs = range(10) 
def processInput(i):
    return i * i
 
num_cores = multiprocessing.cpu_count()

def _pw3( x ):
    a = pow(abs(x),1/3); 
    return a   

pw3 = gpl.functions.make_function( function = _pw3, name='pw3', arity=1 )
function_set = ['add', 'sub', 'mul', 'div',pw3]
M = 100

#os.remove("output.txt")
inputs = range(M)
def processInput(i):
  est_gp = SymbolicRegressor(population_size=40000,generations=50,
                           stopping_criteria=0.02,
                           p_crossover=0.65, p_subtree_mutation=0.15,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           verbose=0,n_jobs=1,const_range=(-0.1,0.1),
                           function_set=function_set,metric='rmse',
                           parsimony_coefficient=0.1, random_state=i)
  est_gp.fit(X_train, y_train)
  print(est_gp._program,file=open("output.txt","a"))
  y_score = est_gp.predict(X_train)
  rmse = np.sqrt(np.sum(np.power(y_train-y_score,2))/len(y_train))
  return rmse

rmse = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)


Populating the interactive namespace from numpy and matplotlib


FileNotFoundError: File b'exa1.csv' does not exist

In [2]:
!pip3 install gplearn

    100% |████████████████████████████████| 51kB 387kB/s ta 0:00:011
    100% |████████████████████████████████| 296kB 655kB/s eta 0:00:01
    100% |████████████████████████████████| 7.0MB 649kB/s eta 0:00:01   8% |██▊                             | 604kB 23.0MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.19.1
    Uninstalling scikit-learn-0.19.1:
      Successfully uninstalled scikit-learn-0.19.1
You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
